In [85]:
from binance.client import Client
import pandas as pd
import requests
from datetime import datetime, timedelta
import datetime
import requests
import pandas as pd
import hopsworks
import datetime
import os
import warnings
warnings.filterwarnings("ignore")

In [86]:

api_key = 'ZkndW133Kc7JBCDbVV5DV9TSkpkTAm93VaKOSygIDiKi6rOK9tKnOubD4gXY1SSs'
api_secret = 'l3QgD9k6dXDcArEhzSnfXCVtpSykCilnPrk3Ol5xjbJuoH2XYfResrjuAeBHoJgZ'

client = Client(api_key, api_secret)
def convert_to_utc(timestamp_ms):
    """
    Convert the timestamp in milliseconds to a UTC datetime object.
    """
    return datetime.datetime.utcfromtimestamp(timestamp_ms / 1000)

In [87]:
def get_last_hour_gold_price(symbol="PAXGUSDT", interval=Client.KLINE_INTERVAL_1HOUR):
    """
    Retrieves the features (open, high, low, close, volume) of the last closed hour for the given symbol.
    """
    try:
      
        klines = client.get_klines(symbol=symbol, interval=interval, limit=2)

        last_closed_kline = klines[-2]
        utc_timestamp = datetime.datetime.utcfromtimestamp(last_closed_kline[0] / 1000)  # Convert to UTC datetime
        stockholm_timestamp = utc_timestamp + datetime.timedelta(hours=2)  # Adjust to UTC+1

        features = {
            "timestamp": stockholm_timestamp.strftime("%Y-%m-%d %H:%M:%S"),
            "open": float(last_closed_kline[1]),
            "high": float(last_closed_kline[2]),
            "low": float(last_closed_kline[3]),
            "gold_price": float(last_closed_kline[4]),
            "volume": float(last_closed_kline[5])
        }
        
        return features
    except Exception as e:
        print(f"Error fetching last closed hour features: {e}")
        return None


In [88]:
last_features = get_last_hour_gold_price()
print("Last Closed Hour Features:")
print(last_features)


Last Closed Hour Features:
{'timestamp': '2025-01-05 13:00:00', 'open': 2650.0, 'high': 2650.0, 'low': 2644.0, 'gold_price': 2646.0, 'volume': 27.2359}


In [89]:
def get_last_two_closed_hours_data(symbol, interval='1h', timezone_offset=1):

    try:
        # Binance API endpoint for historical klines (candlestick data)
        url = f"https://api.binance.com/api/v3/klines"
        params = {
            'symbol': symbol,
            'interval': interval,
            'limit': 3
        }
        
        response = requests.get(url, params=params)
        
        if response.status_code == 200:
            data = response.json()
            last_closed_kline = data[-2]
            hour_before_klin = data [-3]
            
            def adjust_timestamp(kline):
                utc_timestamp = datetime.datetime.utcfromtimestamp(kline[0] / 1000)
                return utc_timestamp + timedelta(hours=2)
            
            last_hour_data = {
                "timestamp": adjust_timestamp(last_closed_kline).strftime("%Y-%m-%d %H:%M:%S"),
                "open": float(last_closed_kline[1]),
                "high": float(last_closed_kline[2]),
                "low": float(last_closed_kline[3]),
                "close": float(last_closed_kline[4]),
                "volume": float(last_closed_kline[5])
            }
            hour_before_data = {
                "timestamp": adjust_timestamp(hour_before_klin).strftime("%Y-%m-%d %H:%M:%S"),
                "open": float(hour_before_klin[1]),
                "high": float(hour_before_klin[2]),
                "low": float(hour_before_klin[3]),
                "close": float(hour_before_klin[4]),
                "volume": float(hour_before_klin[5])
            }
            
            return hour_before_data, last_hour_data
        else:
            print(f"Error fetching data for {symbol}: {response.status_code} - {response.text}")
            return None
    except Exception as e:
        print(f"An error occurred for {symbol}: {e}")
        return None


In [90]:
def calculate_inflation_rate(symbol, interval='1h', timezone_offset=1):

    hour_before, last_data = get_last_two_closed_hours_data(symbol, interval, timezone_offset)
    
    if not hour_before or not last_data:
        print("Unable to fetch data for inflation calculation.")
        return None
    
    # Calculate inflation rate
    second_last_close = hour_before["close"]
    last_close = last_data["close"]
    
    if second_last_close > 0:
        inflation_rate = ((last_close - second_last_close) / second_last_close) * 100
    else:
        inflation_rate = None
    
    # Return the result
    return inflation_rate
    


In [91]:
eur_inflation = calculate_inflation_rate(symbol='EURUSDT')
if eur_inflation:
    print("EUR/USDT Inflation Rate:")
    print(eur_inflation)

EUR/USDT Inflation Rate:
-0.01937796725123321
